In [1]:
# Import required libraries
import pandas as pd

In [2]:
# Load circuits.csv from the ergast dataset
circuits = pd.read_csv('ergast/circuits.csv')
circuits[['circuitId', 'name', 'location', 'country', 'lat', 'lng']]

,circuitId,name,location,country,lat,lng
0,1,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800
1,2,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800
2,3,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060
3,4,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111
4,5,Istanbul Park,Istanbul,Turkey,40.95170,29.40500
...,...,...,...,...,...,...
72,75,Autódromo Internacional do Algarve,Portimão,Portugal,37.22700,-8.62670
73,76,Autodromo Internazionale del Mugello,Mugello,Italy,43.99750,11.37190
74,77,Jeddah Corniche Circuit,Jeddah,Saudi Arabia,21.63190,39.10440
75,78,Losail International Circuit,Al Daayen,Qatar,25.49000,51.45420


In [3]:
from meteostat import Stations

def find_nearest_station(lat, lon):
 stations = Stations()
 stations = stations.nearby(lat, lon)
 station = stations.fetch(1)
 return station.index[0] if not station.empty else None

circuits['meteostat_station'] = circuits.apply(
 lambda row: find_nearest_station(row['lat'], row['lng']), axis=1
)
circuits[['circuitId', 'name', 'country', 'meteostat_station']]

,circuitId,name,country,meteostat_station
0,1,Albert Park Grand Prix Circuit,Australia,94865
1,2,Sepang International Circuit,Malaysia,48647
2,3,Bahrain International Circuit,Bahrain,41150
3,4,Circuit de Barcelona-Catalunya,Spain,LELL0
4,5,Istanbul Park,Turkey,LTFJ0
...,...,...,...,...
72,75,Autódromo Internacional do Algarve,Portugal,08538
73,76,Autodromo Internazionale del Mugello,Italy,16170
74,77,Jeddah Corniche Circuit,Saudi Arabia,41024
75,78,Losail International Circuit,Qatar,41161


In [7]:
# Download hourly weather data for each circuit's nearest station (2018-2025) and save to meteostat/*.csv

from meteostat import Hourly
from datetime import datetime
import os
from tqdm import tqdm

# Ensure output directory exists
os.makedirs('meteostat', exist_ok=True)

start = datetime(2018, 1, 1)
end = datetime(2025, 12, 31)

for idx, row in tqdm(circuits.iterrows()):
    station_id = row['meteostat_station']
    circuit_id = row['circuitId']
    if pd.notnull(station_id):
        data = Hourly(station_id, start, end).fetch()
        if not data.empty:
            data.to_csv(f'meteostat/{circuit_id}_{station_id}_2018_2025.csv')
        else:
            print(f"No data for {circuit_id} ({station_id})")
    else:
        print(f"No station found for {circuit_id}")

30it [00:06,  7.20it/s]

No data for 30 (FAGC0)


47it [00:11,  3.75it/s]

No data for 48 (128HL)
No data for 49 (08180)


66it [00:15,  3.87it/s]

No data for 67 (08180)


71it [00:16,  4.17it/s]

No data for 73 (37850)


77it [00:17,  4.29it/s]
